In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import html5lib
import psycopg2
import pickle

In [2]:
con = None
con = psycopg2.connect(database='mountainproject', user='User')
cur = con.cursor()
con.autocommit = True

In [ ]:

def get_user_id_gte(table='stars', n=5):
    #user_ids from users greater than or equal to n reviews
    user_id_gte= '''
        SELECT user_id
        FROM {}
        GROUP BY user_id
        HAVING COUNT(user_id) >= %s;
        '''.format(table)
    cur.execute(user_id_gte, (n,) )
    tup_list = cur.fetchall()
    #return tup_list
    return [tup[0] for tup in tup_list]
#user_id_list = get_user_id_gte('stars', 5), len(user_id_list)


In [5]:
def get_climb_id_from_user_id_gte(id_tuple, n, table='stars'):
    climb_ids = '''
        SELECT climb_id
        FROM {}
        WHERE user_id IN %s
        GROUP BY climb_id
        HAVING COUNT(climb_id) >= %s;
    '''.format(table)
    cur.execute(climb_ids, (id_tuple, n) )
    return [tup[0] for tup in cur.fetchall()]
#climb_id_list = get_climb_id_from_user_id_gte(tuple(user_id_list), 5), len(climb_id_list)


13694

In [6]:
def get_user_id_from_climb_id_gte(id_tuple, n, table='stars'):
    #Get user ids with at least n reviews in the climb id tuple
    user_ids_sql = '''
        SELECT user_id
        FROM {}
        WHERE climb_id IN %s
        GROUP BY user_id
        HAVING COUNT(user_id) >= %s;
    '''.format(table)
    cur.execute(user_ids_sql, (id_tuple, n) )
    return [tup[0] for tup in cur.fetchall()]
#user_id_list2 = get_user_id_from_climb_id_gte(tuple(climb_id_list), 5), len(user_id_list2)


7565

In [30]:
def find_climb_user_ids(table='stars', n=5):
    climb_id_old = [0]
    climb_id_new = [-2]
    user_id_old = [0]
    user_id_new = [-2] 
    user_id_new = get_user_id_gte(table=table, n=n)
    while (set(user_id_old) != set(user_id_new)) and \
            (set(climb_id_old) != set(climb_id_new)):
        print "climb list:", len(climb_id_new)
        print "user list:", len(user_id_new)
        climb_id_old = climb_id_new
        climb_id_new = get_climb_id_from_user_id_gte(tuple(user_id_new), 
                                                     n,
                                                     table=table)
        user_id_old = user_id_new
        user_id_new = get_user_id_from_climb_id_gte(tuple(climb_id_new),
                                                    n,
                                                    table=table)
        
    return user_id_new, climb_id_new

#(user_id_list, climb_id_list) = find_climb_user_ids(table='stars', n=5)

 climb list: 1
user list: 7592
climb list: 13694
user list: 7565
climb list: 13678
user list: 7562
climb list: 13676
user list: 7561


In [18]:
def pickle_id_list(id_list, name):
    # Save id list for future use
    with open('id_list_'+name+'.p','w') as f:
        pickle.dump(id_list, f)

In [36]:
pickle_id_list(climb_id_list, 'climb_star5')
pickle_id_list(user_id_list, 'user_star5')

In [34]:
(user_id_list_sub, climb_id_list_sub) = find_climb_user_ids(n=30)

climb list: 1
user list: 7592
climb list: 3196
user list: 1736
climb list: 2149
user list: 1456
climb list: 1972
user list: 1386
climb list: 1912
user list: 1364
climb list: 1897
user list: 1352
climb list: 1884
user list: 1350
climb list: 1879
user list: 1344
climb list: 1871
user list: 1337
climb list: 1864
user list: 1331
climb list: 1860
user list: 1329
climb list: 1857
user list: 1327
climb list: 1854
user list: 1326


In [35]:
pickle_id_list(climb_id_list_sub, 'climb_star30')
pickle_id_list(user_id_list_sub, 'user_star30')

In [39]:
(user_id_list_sub, climb_id_list_sub) = find_climb_user_ids(n=35)

climb list: 1
user list: 7592
climb list: 2639
user list: 1403
climb list: 1560
user list: 1019
climb list: 1271
user list: 872
climb list: 1117
user list: 793
climb list: 1040
user list: 754
climb list: 996
user list: 729
climb list: 975
user list: 719
climb list: 966
user list: 716
climb list: 959
user list: 712
climb list: 951
user list: 706
climb list: 944
user list: 702
climb list: 942
user list: 701
climb list: 940
user list: 700


In [ ]:
pickle_id_list(climb_id_list_sub, 'climb_star35')
pickle_id_list(user_id_list_sub, 'user_star35')